In [32]:
#todo -- reorder the lines to be in a more logical order
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.simplefilter('ignore')


In [2]:
df = pd.read_csv('data_clean.csv')

In [3]:
df.head()

,Unnamed: 0,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
df['koi_disposition'].unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [5]:
dfy_ = df['koi_disposition']
dfy_ = dfy_.values.reshape(-1, 1)
label = LabelEncoder()
dfy = label.fit(dfy_).transform(dfy_)


In [6]:
y_labels = label.classes_


In [7]:
df = df.drop(columns=['Unnamed: 0'])
#todo - don't save the index on the to_csv function in the cleaning notebook

In [8]:
dfx = df.drop(columns=['koi_disposition'])

In [9]:
dfx.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [10]:
y_labels_df = pd.DataFrame({'label': y_labels})
y_labels_df

,label
0,CANDIDATE
1,CONFIRMED
2,FALSE POSITIVE


In [11]:
print(dfx.shape, dfy.shape)
#if the y shape causes problems, I can put into a DF and reshape

(6991, 40) (6991,)


In [12]:
#split data into training and testing data
x_train, x_test, y_train, y_test = train_test_split(dfx, dfy, random_state=55)

In [13]:
#scale x values
x_scaler = StandardScaler().fit(x_train)
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)



In [15]:
model = RandomForestClassifier(n_estimators=200)


In [16]:
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [17]:
#score the model with test data
model.score(x_test, y_test)

0.8918764302059496

In [31]:
#see the score with training data
model.score(x_train, y_train)

1.0

In [30]:
#predicts the koi_disposition of the 354 index of the df
res = model.predict(dfx.loc[354,].values.reshape(1, -1))[0]
#look up the corresponding string classification associated with the numerical classification
y_labels_df.loc[res,]['label']

'FALSE POSITIVE'

In [47]:
#get value of each feature in a list
feature_value = list(model.feature_importances_)
#list of feature names
feature_names = list(dfx.columns)

In [48]:
#loop to zip the feature name and its value together
features_and_values = [(feature, importance) for feature, importance in zip(feature_names, feature_value)]
features_and_values

[('koi_fpflag_nt', 0.09997309875062023),
 ('koi_fpflag_ss', 0.07060797267309098),
 ('koi_fpflag_co', 0.1104767737922937),
 ('koi_fpflag_ec', 0.036295014385215964),
 ('koi_period', 0.022233303450640084),
 ('koi_period_err1', 0.017661897261969418),
 ('koi_period_err2', 0.01760047048706967),
 ('koi_time0bk', 0.012474941426753634),
 ('koi_time0bk_err1', 0.019891497694823658),
 ('koi_time0bk_err2', 0.023280638725144777),
 ('koi_impact', 0.01724070625213356),
 ('koi_impact_err1', 0.010576426905618587),
 ('koi_impact_err2', 0.01064441492976345),
 ('koi_duration', 0.023007426873896956),
 ('koi_duration_err1', 0.03369579842843987),
 ('koi_duration_err2', 0.03267520738099935),
 ('koi_depth', 0.018948586764137222),
 ('koi_depth_err1', 0.015763112601120576),
 ('koi_depth_err2', 0.013822860004982033),
 ('koi_prad', 0.051279406046989225),
 ('koi_prad_err1', 0.029397363353831175),
 ('koi_prad_err2', 0.029865195801542163),
 ('koi_teq', 0.01767311460475142),
 ('koi_insol', 0.015202847053243342),
 ('koi

In [50]:
sorted(features_and_values, key = lambda x: x[1], reverse = True) 

[('koi_fpflag_co', 0.1104767737922937),
 ('koi_fpflag_nt', 0.09997309875062023),
 ('koi_fpflag_ss', 0.07060797267309098),
 ('koi_model_snr', 0.06013885873535871),
 ('koi_prad', 0.051279406046989225),
 ('koi_fpflag_ec', 0.036295014385215964),
 ('koi_duration_err1', 0.03369579842843987),
 ('koi_duration_err2', 0.03267520738099935),
 ('koi_prad_err2', 0.029865195801542163),
 ('koi_prad_err1', 0.029397363353831175),
 ('koi_steff_err1', 0.028281279446738607),
 ('koi_steff_err2', 0.024682204387982166),
 ('koi_time0bk_err2', 0.023280638725144777),
 ('koi_duration', 0.023007426873896956),
 ('koi_period', 0.022233303450640084),
 ('koi_time0bk_err1', 0.019891497694823658),
 ('koi_depth', 0.018948586764137222),
 ('koi_teq', 0.01767311460475142),
 ('koi_period_err1', 0.017661897261969418),
 ('koi_period_err2', 0.01760047048706967),
 ('koi_impact', 0.01724070625213356),
 ('koi_insol_err1', 0.017212543523166188),
 ('koi_depth_err1', 0.015763112601120576),
 ('koi_insol', 0.015202847053243342),
 ('koi

In [51]:
#create a copy of dfx
dfx_important_features = dfx.copy()
#drop columns of features with importance value of less than 1%
dfx_important_features = dfx_important_features.drop(columns=['koi_slogg', 'koi_srad', 'koi_steff', 'koi_slogg_err1',
                                                             'koi_srad_err2', 'koi_tce_plnt_num'])

In [52]:
#split data into testing and training data
#the random state remained at 55
x_train, x_test, y_train, y_test = train_test_split(dfx_important_features, dfy, random_state=55)

In [53]:
#create new model
model_important_features = RandomForestClassifier(n_estimators=200)


In [54]:
#train new model
model_important_features.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [55]:
#score new model with testing data
model_important_features.score(x_test, y_test)
#didn't help very much, the r^2 only went up by about 0.003

0.8953089244851259

In [56]:
model_important_features.score(x_train, y_train)

1.0

In [79]:
max_depth = list(np.arange(50, 201, 25))
max_depth.append(None)
param_grid = {
    'bootstrap': [True, False], 
    'n_estimators': list(np.arange(100, 301, 50)),
    'max_features': ['auto', 'log2', None],
    'max_depth': max_depth,
    'n_jobs': [3]
}
grid = GridSearchCV(model_important_features, param_grid, verbose=3)

In [80]:
grid.fit(x_train, y_train)

Fitting 3 folds for each of 240 candidates, totalling 720 fits
[CV] bootstrap=True, max_depth=50, max_features=auto, n_estimators=100, n_jobs=3 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=True, max_depth=50, max_features=auto, n_estimators=100, n_jobs=3, score=0.897, total=   1.8s
[CV] bootstrap=True, max_depth=50, max_features=auto, n_estimators=100, n_jobs=3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  bootstrap=True, max_depth=50, max_features=auto, n_estimators=100, n_jobs=3, score=0.888, total=   0.5s
[CV] bootstrap=True, max_depth=50, max_features=auto, n_estimators=100, n_jobs=3 


...

[CV]  bootstrap=False, max_depth=175, max_features=auto, n_estimators=300, n_jobs=3, score=0.885, total=   1.7s
[CV] bootstrap=False, max_depth=175, max_features=log2, n_estimators=100, n_jobs=3 
[CV]  bootstrap=False, max_depth=175, max_features=log2, n_estimators=100, n_jobs=3, score=0.892, total=   0.8s
[CV] bootstrap=False, max_depth=175, max_features=log2, n_estimators=100, n_jobs=3 
[CV]  bootstrap=False, max_depth=175, max_features=log2, n_estimators=100, n_jobs=3, score=0.891, total=   0.7s
[CV] bootstrap=False, max_depth=175, max_features=log2, n_estimators=100, n_jobs=3 
[CV]  bootstrap=False, max_depth=175, max_features=log2, n_estimators=100, n_jobs=3, score=0.884, total=   0.7s
[CV] bootstrap=False, max_depth=175, max_features=log2, n_estimators=150, n_jobs=3 
[CV]  bootstrap=False, max_depth=175, max_features=log2, n_estimators=150, n_jobs=3, score=0.895, total=   0.9s
[CV] bootstrap=False, max_depth=175, max_features=log2, n_estimators=150, n_jobs=3 
[CV]  bootstrap=Fals

[CV]  bootstrap=False, max_depth=200, max_features=auto, n_estimators=250, n_jobs=3, score=0.888, total=   1.4s
[CV] bootstrap=False, max_depth=200, max_features=auto, n_estimators=300, n_jobs=3 
[CV]  bootstrap=False, max_depth=200, max_features=auto, n_estimators=300, n_jobs=3, score=0.893, total=   1.7s
[CV] bootstrap=False, max_depth=200, max_features=auto, n_estimators=300, n_jobs=3 
[CV]  bootstrap=False, max_depth=200, max_features=auto, n_estimators=300, n_jobs=3, score=0.890, total=   1.8s
[CV] bootstrap=False, max_depth=200, max_features=auto, n_estimators=300, n_jobs=3 
[CV]  bootstrap=False, max_depth=200, max_features=auto, n_estimators=300, n_jobs=3, score=0.885, total=   1.6s
[CV] bootstrap=False, max_depth=200, max_features=log2, n_estimators=100, n_jobs=3 
[CV]  bootstrap=False, max_depth=200, max_features=log2, n_estimators=100, n_jobs=3, score=0.889, total=   0.8s
[CV] bootstrap=False, max_depth=200, max_features=log2, n_estimators=100, n_jobs=3 
[CV]  bootstrap=Fals

[CV]  bootstrap=False, max_depth=None, max_features=auto, n_estimators=200, n_jobs=3, score=0.881, total=   1.2s
[CV] bootstrap=False, max_depth=None, max_features=auto, n_estimators=250, n_jobs=3 
[CV]  bootstrap=False, max_depth=None, max_features=auto, n_estimators=250, n_jobs=3, score=0.894, total=   1.4s
[CV] bootstrap=False, max_depth=None, max_features=auto, n_estimators=250, n_jobs=3 
[CV]  bootstrap=False, max_depth=None, max_features=auto, n_estimators=250, n_jobs=3, score=0.890, total=   1.3s
[CV] bootstrap=False, max_depth=None, max_features=auto, n_estimators=250, n_jobs=3 
[CV]  bootstrap=False, max_depth=None, max_features=auto, n_estimators=250, n_jobs=3, score=0.884, total=   1.4s
[CV] bootstrap=False, max_depth=None, max_features=auto, n_estimators=300, n_jobs=3 
[CV]  bootstrap=False, max_depth=None, max_features=auto, n_estimators=300, n_jobs=3, score=0.893, total=   1.6s
[CV] bootstrap=False, max_depth=None, max_features=auto, n_estimators=300, n_jobs=3 
[CV]  boot

[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed: 27.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=200, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             i

In [85]:
best_parameters = grid.best_params_
best_parameters

{'bootstrap': True,
 'max_depth': 75,
 'max_features': None,
 'n_estimators': 100,
 'n_jobs': 3}

In [86]:
#create 'best' model after hypertuning
model_best = RandomForestClassifier(bootstrap=True, max_depth=75, max_features=None, n_estimators=100)

In [87]:
#fit model
model_best.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=75, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [88]:
#score model
model_best.score(x_test, y_test)

0.8890160183066361

In [89]:
#score with training data
model_best.score(x_train, y_train)

0.9998092695021934

In [90]:
#out of curiousity, tried changing estimators to very large number
model_large_est = RandomForestClassifier(n_estimators=1200)

In [91]:
model_large_est.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [92]:
model_large_est.score(x_test, y_test)

0.8935926773455377

In [93]:
model_large_est.score(x_train, y_train)

1.0